In [3]:
#  Import libraries and define variables
import os
import shutil
import glob
from tqdm import tqdm
from PIL import Image
import nibabel as nib
import numpy as np
import pickle
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dense, Flatten, Dropout, LeakyReLU, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
import tensorflow as tf
import pandas as pd

import sys
sys.path.append('..')
import config

# Define the modalities and classifications
modalities = ['t1', 't1ce', 't2', 'flair']
classifications = ['MGMT_positive', 'MGMT_negative']

# Define patch size and stride
block_h, block_w = (32, 32)
stride = 2

# Interpolated image dimestions
inter_dim = (110, 90)

# Define epoch
epoch = 5
batch_size = 32

# Define paths to the BraTS dataset folders
path = config.DATASET_PATH

PATH = config.MAIN_DIR + 'Data/'
Org_Dir = PATH + 'Original_Data_Backup/'
Work_Dir = PATH + 'Working_Data/'

In [4]:
# Function Defination --> Load the data

def load_data():
    print('Loading data...')
    pickle_in = open(Work_Dir + 'X.pickle', 'rb')
    X = pickle.load(pickle_in) # Loading Features

    pickle_in = open(Work_Dir + 'y.pickle', 'rb')
    y = pickle.load(pickle_in) # Loading Labels 

    return X, y

In [5]:
#  Funtion Defination --> Model defination
def Model(img_h, img_w, X_train, y_train, X_test, y_test, epoch):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
              input_shape=(img_h, img_w, 1)))
    model.add(LeakyReLU(alpha=0.1))
    # Pool_size is the size of filter.
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Flatten())  # Convert 3D feature map to 1D feature vector.

    model.add(Dense(1096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))

    # Compiling the model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    model.optimizer.learning_rate = 0.0001

    # Training the model & storing its detail in the variable h
    history = model.fit(X_train, to_categorical(y_train), batch_size=128, epochs=epoch, verbose=2,
                        validation_data=(X_test, to_categorical(y_test)), shuffle=True)

    # Evaluating performance of the Model
    test_loss, test_acc = model.evaluate(X_test, to_categorical(y_test))
    results.append(test_acc)
    print('Test Loss is: ', test_loss, '\nTest Accuracy is: ', test_acc)

    # Saving the model in hierarchical Data Formate (HDF)
    model.save(path + 'Outputs/latest_model'+'.h5')

    return history, model



In [ ]:
#  Funtion Defination --> Model defination
def Model_datagen(img_h, img_w, train_dataset, val_dataset, epoch):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
              input_shape=(img_h, img_w, 1)))
    model.add(LeakyReLU(alpha=0.1))
    # Pool_size is the size of filter.
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Flatten())  # Convert 3D feature map to 1D feature vector.

    model.add(Dense(1096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))

    # Compiling the model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    model.optimizer.learning_rate = 0.0001

    # Training the model & storing its detail in the variable h
    history = model.fit(train_dataset, batch_size=128, epochs=epoch, verbose=2,
                        validation_data=(val_dataset), shuffle=True)

    # Evaluating performance of the Model
    # test_loss, test_acc = model.evaluate(X_test, to_categorical(y_test))
    # results.append(test_acc)
    # print('Test Loss is: ', test_loss, '\nTest Accuracy is: ', test_acc)

    # Saving the model in hierarchical Data Formate (HDF)
    model.save(config.MAIN_DIR + 'Outputs/latest_model'+'.h5')
    pickle.dump(history, open(config.MAIN_DIR + 'Outputs/history.pickle', 'wb'))

    return history, model



In [6]:
# Function Definition --> For Saving Validation Accuracy in the file

def Saving_Test_Accuracy():
    import numpy as np

    print('Saving Test Accuracy...')

    file = open(Result_File, "a")
    file.write(case)
    file.write(':\n======')
    file.write('\nAll Test Accuracy:\t')

    for ans in results:
        file.write(str(ans) + ', ')

    file.write('\nMean Test Accuracy:\t' + str(np.mean(results)))
    file.write('\nMin Test Accuracy:\t' + str(np.min(results)))
    file.write('\nMax Test Accuracy:\t' + str(np.max(results)))
    file.write('\n\n\n')
    file.close()


In [7]:
# Main cell to execute the functions

X, y = load_data()

results = []

# Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Training the model
history, model = Model(block_h, block_w, X_train, y_train, X_test, y_test, epoch)

Loading data...
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-05-01 03:33:17.524804: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-01 03:33:17.526282: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/5


2023-05-01 03:33:21.728188: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-01 03:33:22.202597: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-01 03:37:41.747620: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


15036/15036 - 270s - loss: 0.6035 - accuracy: 0.6581 - val_loss: 0.5175 - val_accuracy: 0.7305 - 270s/epoch - 18ms/step
Epoch 2/5
15036/15036 - 220s - loss: 0.4943 - accuracy: 0.7496 - val_loss: 0.4658 - val_accuracy: 0.7674 - 220s/epoch - 15ms/step
Epoch 3/5
15036/15036 - 215s - loss: 0.4260 - accuracy: 0.7947 - val_loss: 0.3610 - val_accuracy: 0.8334 - 215s/epoch - 14ms/step
Epoch 4/5
15036/15036 - 213s - loss: 0.3810 - accuracy: 0.8221 - val_loss: 0.3340 - val_accuracy: 0.8473 - 213s/epoch - 14ms/step
Epoch 5/5
15036/15036 - 212s - loss: 0.3484 - accuracy: 0.8407 - val_loss: 0.2788 - val_accuracy: 0.8777 - 212s/epoch - 14ms/step
6683/6683 [==============================] - 35s 5ms/step - loss: 0.2788 - accuracy: 0.8777
Test Loss is:  0.27877816557884216 
Test Accuracy is:  0.8777450323104858


In [ ]:
# new main function

# Create Image Data Generator 
image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

df = pd.read_csv('step2_file_paths.csv')

# Split the data into train and test
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)


training_set = image_gen.flow_from_dataframe(dataframe= train_df,
                                                 x_col="images", y_col="labels",
                                                 class_mode="binary",
                                                 target_size=(block_h,block_w), batch_size=batch_size)
    
validation_set = image_gen.flow_from_dataframe(dataframe=test_df,
                                                 x_col="images", y_col="labels",
                                                 class_mode="binary",
                                                 target_size=(block_h,block_w), batch_size=batch_size)

# Training the model
history, model = Model_datagen(block_h, block_w, training_set, validation_set, epoch)